In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [3]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['Hour','DHI','DNI','WS','RH','T']])
df_test[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['Hour','DHI','DNI','WS','RH','T']])

In [6]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

In [8]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [9]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [14]:
model7 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    #layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

model8 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    #layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [11]:
from sklearn.model_selection import train_test_split

X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(Day, Day7, test_size=0.25, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(Day, Day8, test_size=0.25, random_state=0)

X_train_1.shape, X_valid_1.shape, Y_train_1.shape, Y_valid_1.shape

((39348, 7), (13116, 7), (39348,), (13116,))

In [12]:
model7.compile(loss='mse', optimizer='adam')
model7.fit(X_train_1, Y_train_1, epochs=25, batch_size=48, validation_split=0.25)
model7.evaluate(X_valid_1, Y_valid_1)

Epoch 1/25
615/615 [==============================] - 1s 2ms/step - loss: 261.3701 - val_loss: 163.0298
Epoch 2/25
615/615 [==============================] - 1s 1ms/step - loss: 146.1109 - val_loss: 148.9428
Epoch 3/25
615/615 [==============================] - 1s 1ms/step - loss: 139.2640 - val_loss: 142.4870
Epoch 4/25
615/615 [==============================] - 1s 1ms/step - loss: 138.5922 - val_loss: 139.3852
Epoch 5/25
615/615 [==============================] - 1s 1ms/step - loss: 131.5805 - val_loss: 136.7924
Epoch 6/25
615/615 [==============================] - 1s 1ms/step - loss: 128.3745 - val_loss: 135.2966
Epoch 7/25
615/615 [==============================] - 1s 1ms/step - loss: 133.7321 - val_loss: 138.0534
Epoch 8/25
615/615 [==============================] - 1s 1ms/step - loss: 131.1845 - val_loss: 135.0704
Epoch 9/25
615/615 [==============================] - 1s 1ms/step - loss: 130.6279 - val_loss: 133.0913
Epoch 10/25
615/615 [==============================] - 1s 1ms/st

128.67286682128906

In [13]:
model8.compile(loss='mse', optimizer='adam')
model8.fit(X_train_2, Y_train_2, epochs=25, batch_size=48, validation_split=0.25)
model8.evaluate(X_valid_2, Y_valid_2)

Epoch 1/25
615/615 [==============================] - 1s 2ms/step - loss: 261.1760 - val_loss: 165.4998
Epoch 2/25
615/615 [==============================] - 1s 1ms/step - loss: 161.1089 - val_loss: 152.7731
Epoch 3/25
615/615 [==============================] - 1s 1ms/step - loss: 146.2737 - val_loss: 154.8672
Epoch 4/25
615/615 [==============================] - 1s 1ms/step - loss: 151.3259 - val_loss: 144.7120
Epoch 5/25
615/615 [==============================] - 1s 1ms/step - loss: 145.8187 - val_loss: 144.1693
Epoch 6/25
615/615 [==============================] - 1s 1ms/step - loss: 144.7695 - val_loss: 141.1092
Epoch 7/25
615/615 [==============================] - 1s 1ms/step - loss: 141.2745 - val_loss: 140.4898
Epoch 8/25
615/615 [==============================] - 1s 1ms/step - loss: 143.9331 - val_loss: 140.1232
Epoch 9/25
615/615 [==============================] - 1s 1ms/step - loss: 138.5019 - val_loss: 143.8886
Epoch 10/25
615/615 [==============================] - 1s 1ms/st

134.9080047607422

In [15]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4482 - val_loss: 1.5954
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4298 - val_loss: 1.5940
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4079 - val_loss: 1.5833
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4108 - val_loss: 1.5864
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4088 - val_loss: 1.5688
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3946 - val_loss: 1.5697
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3634 - val_loss: 1.5534
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3682 - val_loss: 1.5492
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3701 - val_loss: 1.5448
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3510 - val_loss: 1.5726

820/820 [==============================] - 1s 1ms/step - loss: 2.5416 - val_loss: 3.0379
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5492 - val_loss: 2.9728
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5557 - val_loss: 3.0059
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5127 - val_loss: 2.9765
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5370 - val_loss: 2.9723
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5331 - val_loss: 2.9691
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5054 - val_loss: 2.9851
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5236 - val_loss: 2.9729
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5120 - val_loss: 2.9752
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5488 - val_loss: 2.9631
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.4551 - val_loss: 2.9257
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4253 - val_loss: 2.9451
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4622 - val_loss: 2.9396
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4659 - val_loss: 2.9773
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5354 - val_loss: 2.9555
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4340 - val_loss: 2.9674
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5192 - val_loss: 2.9422
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4609 - val_loss: 2.9417
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4933 - val_loss: 2.9627
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4409 - val_loss: 2.9632
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.8259 - val_loss: 2.1491
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7711 - val_loss: 2.1858
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8436 - val_loss: 2.1288
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7970 - val_loss: 2.1731
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7877 - val_loss: 2.1572
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8285 - val_loss: 2.2004
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8448 - val_loss: 2.2178
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7966 - val_loss: 2.1786
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7904 - val_loss: 2.1653
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8093 - val_loss: 2.1624
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7318 - val_loss: 0.8797
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7176 - val_loss: 0.8772
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7234 - val_loss: 0.8840
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7199 - val_loss: 0.8729
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7214 - val_loss: 0.8699
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7075 - val_loss: 0.8540
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7232 - val_loss: 0.8560
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7178 - val_loss: 0.8535
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7244 - val_loss: 0.8780
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7231 - val_loss: 0.8516
Ep

(3888, 9)

In [16]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/25
820/820 [==============================] - 2s 2ms/step - loss: 1.4609 - val_loss: 1.6471
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4276 - val_loss: 1.6258
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4115 - val_loss: 1.6183
Epoch 4/25
820/820 [==============================] - 1s 2ms/step - loss: 1.4253 - val_loss: 1.6195
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4148 - val_loss: 1.6118
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4162 - val_loss: 1.5962
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4424 - val_loss: 1.5849
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4206 - val_loss: 1.6040
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4054 - val_loss: 1.5716
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3990 - val_loss: 1.5807

820/820 [==============================] - 1s 1ms/step - loss: 2.6542 - val_loss: 3.0597
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6719 - val_loss: 3.0985
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6800 - val_loss: 3.0163
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6368 - val_loss: 3.0631
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6830 - val_loss: 3.0600
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6260 - val_loss: 3.0566
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6797 - val_loss: 3.0594
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6636 - val_loss: 3.0953
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6936 - val_loss: 3.0846
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6477 - val_loss: 3.0790
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.6821 - val_loss: 3.0944
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6311 - val_loss: 3.1131
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6314 - val_loss: 3.0871
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6461 - val_loss: 3.1390
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6270 - val_loss: 3.1058
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6467 - val_loss: 3.1400
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6465 - val_loss: 3.0739
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5851 - val_loss: 3.0838
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5911 - val_loss: 3.0970
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6478 - val_loss: 3.0869
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.9106 - val_loss: 2.2539
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.9031 - val_loss: 2.2427
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.9259 - val_loss: 2.2570
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8827 - val_loss: 2.2620
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.9063 - val_loss: 2.2704
Epoch 12/25
820/820 [==============================] - 1s 2ms/step - loss: 1.8830 - val_loss: 2.2770
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8707 - val_loss: 2.2594
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8957 - val_loss: 2.2652
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.9512 - val_loss: 2.2835
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.9165 - val_loss: 2.2468
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7555 - val_loss: 0.9080
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7750 - val_loss: 0.9230
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7652 - val_loss: 0.8989
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7383 - val_loss: 0.9003
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7480 - val_loss: 0.9096
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7509 - val_loss: 0.8957
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7359 - val_loss: 0.8988
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7573 - val_loss: 0.9234
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7497 - val_loss: 0.9268
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7640 - val_loss: 0.9302
Ep

(3888, 9)

In [17]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.091529,0.058653,-0.003468,-0.077582,0.021737,0.011525,0.073995,-0.062168,-0.007692
1,0.csv_Day7_0h30m,-0.088430,0.066005,-0.003138,-0.073081,0.023824,0.015210,0.076589,-0.063994,-0.012471
2,0.csv_Day7_1h00m,-0.077024,0.070626,-0.000599,-0.057499,0.021403,0.010255,0.079147,-0.066835,-0.020874
3,0.csv_Day7_1h30m,-0.081011,0.066011,-0.001187,-0.059581,0.020968,0.009071,0.076985,-0.065937,-0.018924
4,0.csv_Day7_2h00m,-0.071860,0.059089,0.001246,-0.047352,0.016366,0.001245,0.075632,-0.066267,-0.017937
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,-0.002024,-0.007178,-0.005490,-0.005653,-0.005252,-0.006486,-0.004876,-0.008379,-0.004589
7772,80.csv_Day8_22h00m,-0.001816,-0.007516,-0.005609,-0.005795,-0.005046,-0.006584,-0.004407,-0.008233,-0.004453
7773,80.csv_Day8_22h30m,-0.001766,-0.007585,-0.005544,-0.005818,-0.005019,-0.006560,-0.004364,-0.008208,-0.004484
7774,80.csv_Day8_23h00m,-0.002473,-0.007287,-0.005730,-0.006056,-0.005155,-0.006703,-0.004160,-0.008062,-0.004513


In [18]:
submission.to_csv('./submission/submission_20210114-2.csv', index=False)